 # COMP90014 Assignment 3
### Semester 2, 2024
Version 1.0 Last edited 25/09/2024.

In [ ]:
### Fill in your student details here
NAME = "Ching-Yi Peng"
ID = "1545824"

## Completing the assignment

***Academic integrity***

This assignment should be completed by each student individually. <br>
Make sure you read this entire document, and ask for help if anything is not clear. <br>
Any changes or clarifications to this document will be announced via the LMS. <br>

Please make sure you review the University's rules on academic honesty and plagiarism: https://academichonesty.unimelb.edu.au/

Do not copy any code from other students or from the internet. This is considered plagiarism.

***Completing the assignment & submission***

To complete the assignment, finish the coding and short answer tasks in this notebook.<br>
You are free to use any IDE to work on the assignment, but only use jupyter to prepare your final submission (ensures compatability with grading software). <br>
Please do not copy or delete cells in this notebook, as this may interrupt the autograding of hidden tests. <br>
Your completed notebook file containing all your answers will be turned in via LMS. Please also submit an HTML file.

***Visible & hidden tests***

In some cases, we have provided test input and test output that you can use to try out your solutions. <br>
These are visible tests and serve to warn you if you've made a mistake but are **not** exhaustive.

During assessment, there are also hidden tests we run to validate your code. <br>
As you won't see the hidden tests, it's up to you to decide whether your code is correct.

**Remember to save your work early and often.**

## Marking

***Graded cells***

Cells that must be completed to receive marks are clearly labelled with the following text:

`# -- GRADED CELL (N marks) - complete this cell --`

Only add answers to these cells.

Some cells are code cells, in which you must complete the code to solve a problem. <br>
Others are markdown cells, in which you must write your answers to short-answer questions.

***Completing code cells***

You will see the following text in graded code cells.

``` python
raise NotImplementedError()
```

You must remove the `raise NotImplementedError()` line from the cell, and replace it with your solution. <br>
If you want to import a library or use a helper function, the import statements must be inside the function for that question. <br>
For example:
    
``` python
# -- GRADED CELL (1 mark) - complete this cell --

def question1(param1: str, param2: int) -> float:
    from collections import defaultdict
```

Include code comments in your solutions! <br>
Well commented code can help you to receive partial marks even if the final solution is incorrect.

***Editing the notebook***

Only graded cells will be marked.
- Do **NOT** enter solutions outside of graded cells
- Do **NOT** duplicate or remove cells from the notebook
- You may add new cells to test code, but new cells will not be graded.
- Word limits, where stated, will be strictly enforced. Answers exceeding the limit **will not be marked**.

***Marks***

The total marks for the assignment add up to 15. <br>
This assignment will be worth 15% of your overall subject grade.



## Submission

Only use jupyter to prepare your final submission. This ensures compatability with our grading software. <br>

Make sure you have filled in any place that says `# -- GRADED CELL (N marks) - complete this cell --`.

Before you turn this assignment in, make sure everything runs as expected, and the output is cleared.

First, **restart the kernel**:

- In the menubar, select Kernel -> Restart.

Next, **run all cells**:

- In the menubar, select Cell -> Run All.

Finally, **clear all output**:

- In the menubar, select Options -> Clear All Outputs

Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format. <br>
You must also submit a copy of this notebook in `html` format with the output cleared.

Your submission should include **only two** files with names formatted as: **Assignment3.ipynb** and **Assignment3.html**



<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Assignment 3</h1>

<h2>De Bruijn Graph Simplification</h2>

</div>

**Structure**

This assignment has two sections.

- In section 1 we will implement a *modified version* of the tour bus algorithm, inspired from the [Velvet](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2336801/) paper.<br>
- In section 2 you need to answer two short-answer questions related to dimensionality reduction and clustering.

**Section 1: The Tour Bus [10 marks]**
- Q1: explore [2 marks]
- Q2: backtrack [2 marks]
- Q3: compare [2 marks]
- Q4: select_consensus [2 marks]
- Q5: collapse [2 marks]
- <span style="color: blue">Q6: collapse_hard [+3 bonus marks] (optional)</span>

**Section 2: Short-answer questions [5 marks]**
- Q7: Dimensionality Reduction [2 marks]
- Q8: Clustering [3 marks]


**Packages**

This assignment uses the <small>`networkx`</small> and <small>`python-levenshtein`</small> packages. <br>
You may use functionality within <small>`networkx`</small> to answer questions in this assignment (if helpful). <br>
These are both installable via pip and conda.


In [ ]:
!pip install python-Levenshtein

In [ ]:
# imports
import networkx as nx
import Levenshtein
from typing import Tuple, List, Optional

**Data**

The following files are required for this assignment:
- https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/nodelist.tsv
- https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/edgelist.tsv
- https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/kmer_counts.tsv

Download the files above and place them in a folder named 'data' relative to your current working directory. <br>
Run the cell below to confirm the paths are correct.

In [ ]:
!mkdir -p data

!wget -P data https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/nodelist.tsv
!wget -P data https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/edgelist.tsv
!wget -P data https://github.com/melbournebioinformatics/COMP90014_2024/raw/master/assignments/data/kmer_counts.tsv

In [ ]:
# NOTE: DO NOT CHANGE THESE PATHS.
# Changing these variables will result in a 1 mark deduction to your grade.
NODELIST = 'data/nodelist.tsv'
EDGELIST = 'data/edgelist.tsv'
KMERCOUNTS = 'data/kmer_counts.tsv'

import os
for filepath in [NODELIST, EDGELIST, KMERCOUNTS]:
    if os.path.exists(filepath):
        print(f'found {filepath}')
    else:
        print(f'error {filepath}')


The graphs used in visible test cases has the same structure as appears in figures.  <br>
A different graph is used for hidden tests. You are encouraged to write new tests so that your code covers edge-cases not present in the visible tests.

Run the cells below to load the nx.DiGraph we will be working with in this assignment.


In [ ]:
# helper function to load graph from file.
def load_graph(nodelist_path: str, edgelist_path: str) -> nx.DiGraph:
    G = nx.DiGraph()

    # read in node data
    with open('data/nodelist.tsv', 'r') as fp:
        lines = fp.readlines()
        lines = [ln.strip().split('\t') for ln in lines]
        for node, seq in lines[1:]:
            G.add_node(node, sequence=seq)

    # read in edge data
    with open('data/edgelist.tsv', 'r') as fp:
        lines = fp.readlines()
        lines = [ln.strip().split('\t') for ln in lines]
        for src, dest, kmer in lines[1:]:
            G.add_edge(src, dest, kmer=kmer)

    # return graph
    return G


In [ ]:
# viewing the graph data
G = load_graph(NODELIST, EDGELIST)

print('\nNODES ---')
print('node\tsequence')
for node, data in G.nodes(data=True):
    print(node + '\t' + data['sequence'])

print('\nEDGES ---')
print('node1\tnode2\tkmer')
for n1, n2, data in G.edges(data=True):
    print(n1 + '\t' + n2 + '\t' + data['kmer'])

<br>

<div style="color: rgb(0,0,0); background: #0096FF; border: solid 1px rgb(0,0,0); padding: 10px;">

<h1>Section 1: The Tour Bus</h1>

</div>


### Introduction

Complex regions in De Bruijn Graphs can be caused by many factors, including read errors, heterozygosity, and repetitive sequence.
To handle such situations, the genome assembler 'Velvet' uses an approach called the 'Tour Bus'.

The main ideas of the Tour Bus are as follows:
- Two branching paths, which start and end at the same node, are termed 'bubbles' due to their topological appearence.
- These paths may encode similar sequence along their length, and if so, may have formed due to sequencing errors or biological variability (eg heterozygosity).  
- In such a situation, these paths can be collapsed into a single 'consensus' path, simplifying the graph (and improving the final assembly).

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/tourbus.png" width="600">

In the figure above we see an example of this situation. <br>
The 'consensus' path is the series of nodes [A, B, C, D, E], with nodes B', C', C'', and D' being caused by the types of errors mentioned above.

You will notice that there are two bubbles in this graph. <br>
As such, bubbles need to be handled iteratively, where we detect and resolve a single bubble at each iteration.



<br>

-----------------

### Identifying Bubbles

The first step of each Tour Bus iteration is to identify whether a bubble exists. <br>
The graph is explored, starting at an arbitrary node, and stopping when a previously visited node is a neighbour of the current node. <br>
In this situation, both the current node and the visited node are returned.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/explore.png" width="600">



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1</h3>

<b>Challenge:</b> Write a function which explores a graph to identify bubbles. <br>

- [ ] Input: G (networkx.DiGraph); node (str).
- [ ] Ouput: A tuple with the format (current node, visited node).
- [ ] If there is no bubble to resolve, return <small>`None`</small>.

When exploring node neighbours, break ties alphabetically. <br>
For example, if the neighbours of node <small>`A`</small> are <small>`B`</small> and <small>`B'`</small>, you should visit <small>`B`</small> first.
    
</div>


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --
from collections import deque

def explore(G: nx.DiGraph, node: str) -> Optional[Tuple]:
    """
    Explores the graph from a provided starting node to identify a bubble.
    Returns the current node and previously visited node as a tuple if present.
    If no bubble is identified, returns None.
    """
    visited = set()
    queue = deque(node)

    while queue:
        current_node = queue.popleft()
        visited.add(current_node)

        # Get neighbors of the current node sorted alphabetically
        neighbors = sorted(G.neighbors(current_node))

        for neighbor in neighbors:
            if not neighbor in visited:
                queue.append(neighbor)
                visited.add(neighbor)
            else:
              return (current_node, neighbor)

    # If no bubble is found
    return None


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - DO NOT ALTER.
G = load_graph(NODELIST, EDGELIST)

# test1: from node A
res = explore(G, 'A')
print(f"test1: expected=('C`', 'D'), actual={res}")

# test2: from node B
res = explore(G, 'B')
print(f"\ntest2: expected=('D', 'E'), actual={res}")

# test3: from node C
res = explore(G, 'C')
print(f"\ntest3: expected=None, actual={res}")


<br>

--------------------

### Backtracking

After a bubble has been identified, the second step is to extract the two paths involved in the bubble.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/backtrack.png" width="600">

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 2</h3>

<b>Challenge:</b> Write a function which uses backtracking to return two paths involved in a bubble.<br>

- [ ] Input: G (networkx.DiGraph); current_node (str); visited_node (str).
- [ ] Ouput: A tuple with the format (path1, path2), where each path is a list of nodes.

Details:
- Backtrack from <small>`current_node`</small> and <small>`visited_node`</small>.
- <small>`visited_node`</small> should appear in both output paths, so that both paths start and end at the same node.
- Ensure output paths read left to right in the figure above. ie node <small>`A`</small> should be the first item, and node <small>`D`</small> should be the last item.

Consider situations where a node has multiple in-edges, for example node <small>`B'`</small> in the figure above.
    
</div>


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def backtrack(G: nx.DiGraph, current_node: str, visited_node: str) -> Tuple[List, List]:
    """
    Performs backtracking to extract two paths (of nodes) involved in a bubble.
    """
    path1 = [current_node, visited_node]
    path2 = [visited_node]
    # Track visited nodes to avoid cycles
    visited = set([current_node])

    # Backtrack from current_node to visited_node to find path1
    while current_node != visited_node:
        predecessors = sorted(G.predecessors(current_node))
        found_predecessor = False
        for pred in predecessors:
            if pred not in visited:
                path1.append(pred)
                current_node = pred
                visited.add(pred)
                found_predecessor = True
                break
        # If no predecessor is found, break the loop to avoid infinite loops
        if not found_predecessor:
            break

    # Reset current_node to visited_node for the second path
    current_node = visited_node
    visited = set([current_node])

    # Backtrack to find an alternate path (path2)
    while current_node != path1[-1]:
        predecessors = sorted(G.predecessors(current_node))
        found_predecessor = False
        for pred in predecessors:
            # Avoid using predecessors already used in path1
            if pred not in visited:
                path2.append(pred)
                current_node = pred
                visited.add(pred)
                found_predecessor = True
                break
        if current_node in path1:
            # current_index = path1.index(current_node)
            # if current_index > 0:
            #     previous_node = path1[current_index]
            #     path1.remove(previous_node)
            break
        # If no predecessor is found, break the loop to avoid infinite loops
        if not found_predecessor:
            break

    # Reverse paths to maintain left-to-right order
    return (sorted(path1), sorted(path2))


In [ ]:
# Testing cell - DO NOT ALTER.
G = load_graph(NODELIST, EDGELIST)

# test1: as in figure
path1, path2 = backtrack(G, 'C`', 'D')
print('test 1')
print(f"expected: path1 = ['A', 'B`', 'C`', 'D'], path2 = ['A', 'B', 'C', 'D']")
print(f"actual:   path1 = {path1}, path2 = {path2}")

# another bubble
path1, path2 = backtrack(G, 'D', 'E')
print('\ntest 2')
print(f"expected: path1 = ['B', 'C', 'D', 'E'], path2 = ['B', 'C``D`', 'E']")
print(f"actual:   path = {path1}, path2 = {path2}")


<br>

----------------------------

### Comparing sequences

Now that a bubble has been identified and two node paths have been fetched, we need to check how similar the sequences are along these paths.<br>
If highly similar (determined using levenshtein distance), we can confidently assume the bubble was due to biological variation or sequencing error.<br>
If too dissimilar, we should ignore the bubble.

The figure below demonstrates two situations you will need to handle, as the two paths may not have the same number of nodes.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/compare.png" width="500">

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3</h3>

<b>Challenge:</b> Write a function <small>`compare()`</small> which compares the sequence along two paths to decide whether the paths can be merged. <br>

- [ ] Input: G (networkx.DiGraph); path1 (list of str); path2 (list of str); a kmer-size k (int) (which was originally used to generate the De Bruijn Graph).
- [ ] Ouput: True or False (bool).

Details:
- Think about whether the start and end node of each path (shared) should be considered.
- Consider that nodes in De Buijn Graphs are originally <small>`k-1`</small> prefix/suffix pairs, and how this will affect the sequence along each path.
- The sequence for a given node is available via a data attribute named <small>`'sequence'`</small> (eg <small>`G.nodes['A']['sequence']`</small>)
- A function named <small>`get_edits()`</small> has been provided below to return the number of edits between two sequences.
- Decision condition - the number of edits must be less-than or equal-to 20% of the longer sequence for the paths to be considered 'similar'.
    
</div>


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

# helper function
def do_align(seq1: str, seq2: str) -> float:
    return Levenshtein.distance(seq1, seq2)

def compare(G: nx.DiGraph, path1: List[str], path2: List[str], k: int=5) -> bool:
    """
    Compares the sequence along two paths to decide whether they are similar enough to merge.
    The number of edits must be less than 20% of the length of the longer sequence to be considered as 'similar'.
    """
    # Concatenate the sequences of all nodes in path1
    full_seq1 = "".join([G.nodes[node]['sequence'] for node in path1])

    # Concatenate the sequences of all nodes in path2
    full_seq2 = "".join([G.nodes[node]['sequence'] for node in path2])

    # Compute the Levenshtein distance between the full sequences
    edits = do_align(full_seq1, full_seq2)

    # Get the length of the longer sequence
    max_length = max(len(full_seq1), len(full_seq2))

    return edits <= 0.2 * max_length


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - DO NOT ALTER.
G = load_graph(NODELIST, EDGELIST)

# test1: a valid bubble
res = compare(G, ['A', 'B`', 'C`', 'D'], ['A', 'B', 'C', 'D'], k=5)
print(f'test1: expected=True, actual={res}')

# test2: another valid bubble
print()
res = compare(G, ['B', 'C', 'D', 'E'], ['B', 'C``D`', 'E'], k=5)
print(f'test2: expected=True, actual={res}')

# test3: not a bubble (or valid paths for that matter), but valid as a test case.
print()
res = compare(G, ['B`', 'C`', 'D'], ['B`', 'X', 'D'], k=5)
print(f'test3: expected=False, actual={res}')


<br>

---------------------------

### Selecting the consensus (correct) path

Once the two paths have been deemed similar enough to merge, we can collapse the bubble.

To begin, the 'consensus' path must be identified. <br>
This indicates which path is likely correct, and is calculated using kmer counts from the original readset used to construct our De Bruijn Graph. <br>

The De Bruijn Graph we are working with has already been simplified by coalescing linear chains of nodes. <br>
This transformation can be seen in the figure below.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/consensus.png" width="600">


To determine the consensus path, we will calculate a 'path length' for each path in the bubble. <br>
The formula we will use is <small>`len(p) / multiplicity(p)`</small>, where:
- <small>`len(p)`</small> refers to the number of kmers along the path
- <small>`multiplicity(p)`</small> refers to the cumulative counts (multiplicity) of kmers along the path.

These kmer counts are available via the provided <small>`kmer_counts`</small> dictionary which maps kmers to their associated multiplicity (represented as a table in the figure above).<br>
As displayed in the figure above, the path length remains unchanged after linear chains have been coalesced, as kmers can be extracted out of node sequences. <br>

For example:
- The path ['GGAT', 'GATTACA', 'ACAT'] will have a path length of <small>`5 / (10+12+10+11+9) = 0.096`</small>.
- The path ['CGAT', 'GATTACA', 'ACAA'] will have a path length of <small>`5 / (1+12+10+11+2) = 0.139`</small>.

The path with the smaller 'path length' will be selected as the consensus, as prioritises the path with stronger read evidence.


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 4</h3>

<b>Challenge:</b> Write a function which identifies the consensus path from two candidate paths. <br>

- [ ] Input: G (networkx.DiGraph); path1 (list of str); path2 (list of str); kmer_counts (dict); a kmer size k (int).
- [ ] Ouput: One of the input paths (the consensus path).
    
Edge kmers are available via a data attribute named <small>`'kmer'`</small> for a given edge (eg <small>`G.edges['A', 'B']['kmer']`</small>)

</div>


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def select_consensus(G: nx.DiGraph, path1: List[str], path2: List[str], kmer_counts: dict, k: int=5) -> List:
    """
    Selects and returns the consensus path by calculating the path with lower 'path length'.
    """
    def calculate_path_length(path):
        total_kmer_count = 0
        for i in range(len(path) - 1):
            edge_kmer = G.edges[path[i], path[i + 1]].get('kmer')
            if edge_kmer is None:
                continue  # Skip this edge if no k-mer is found

            kmer_count = kmer_counts.get(edge_kmer, 0)
            total_kmer_count += kmer_count

        # Avoid division by zero in case of missing k-mer counts
        if total_kmer_count == 0:
            return float('inf')

        # Calculate path length using the formula
        path_length = k / total_kmer_count
        return path_length

    # Calculate the path lengths for both paths
    path1_length = calculate_path_length(path1)
    path2_length = calculate_path_length(path2)

    # Select the path with the smaller path length
    if path1_length < path2_length:
        return path1
    elif path2_length < path1_length:
        return path2
    else:
        return None


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - DO NOT ALTER.
G = load_graph(NODELIST, EDGELIST)
with open(KMERCOUNTS, 'r') as fp:
    lines = fp.readlines()
    lines = [ln.strip().split('\t') for ln in lines[1:]]
    kmer_counts = {k: int(c) for k, c in lines}

# single test: a valid bubble
print()
path1 = ['A', 'B`', 'C`', 'D']
path2 = ['A', 'B', 'C', 'D']
res = select_consensus(G, path1, path2, kmer_counts, k=5)
print('note: the path length of path1 should be approximately 0.2, and path length of path2 should be approximately 0.1')
print(f"test1: expected=['A', 'B', 'C', 'D'], actual={res}")



<br>

----------------

### Collapsing the Bubble

Finally, we can collapse the bubble. <br>
We will take an iterative approach as displayed in the figure below.

Note that the edge <small>`B' -> X`</small> is regrafted to <small>`B -> X`</small>.<br>
This somewhat violates the concept of edges in a De Bruijn Graph being kmers which connect prefix/suffix pairs, but is required nonetheless.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/collapse.png" width="500">



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 5</h3>

<b>Challenge:</b> Write a function which collapses the graph bubble. <br>

- [ ] Input: G (networkx.DiGraph); consensus_path (list of str); minority_path (list of str).
- [ ] Ouput: Updated graph with the bubble removed (networkx.DiGraph)

</div>


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def collapse(G: nx.DiGraph, consensus_path: List[str], minority_path: List[str]) -> nx.DiGraph:
    """
    Collapses a bubble by removing the minority_path from the graph.
    Returns the simplified graph G with bubble removed.
    """
    for node in minority_path:
        # Only remove the node if it's not part of the consensus path
        if node not in consensus_path:
            if G.has_node(node):
                G.remove_node(node)

    return G

In [ ]:
# extra code cell for development if needed

def collapse_2(G: nx.DiGraph, consensus_path: List[str], minority_path: List[str]) -> nx.DiGraph:
    """
    Collapses a bubble by removing the minority_path from the graph.
    Returns the simplified graph G with bubble removed.
    """
    for node_co in consensus_path:
        for node_min in minority_path:
            if node_co != node_min and G.has_node(node_min):
                G.remove_node(node_min)  # Remove nodes in the minority path

        # Check if the current consensus node exists before accessing its predecessors
        if G.has_node(node_co):
            pre_nodes = list(G.predecessors(node_co))

            # For each predecessor, contract it with the current node in the consensus path
            for pre in pre_nodes:
                if G.has_node(pre):
                    # Contract predecessor into the current consensus node
                    G = nx.contracted_nodes(G, node_co, pre, self_loops=False)

    return G

In [ ]:
# Testing cell - DO NOT ALTER.
G = load_graph(NODELIST, EDGELIST)

# test1: the first bubble
consensus_path = ['A', 'B', 'C', 'D']
minority_path = ['A', 'B`', 'C`', 'D']
G = collapse(G, consensus_path, minority_path)
print("\ntest1 (remaining nodes)")
print("expected: [['AB', 'C', 'C``D`', 'D', 'E', 'X']")
print(f"actual:   {sorted(G.nodes())}")

# test2: the second bubble
consensus_path = ['AB', 'C', 'D', 'E']
minority_path = ['AB', 'C``D`', 'E']
G = collapse(G, consensus_path, minority_path)
print("\ntest2 (remaining nodes)")
print("expected: ['AB', 'CDE', 'X']")
print(f"actual:   {sorted(G.nodes())}")


<br>

----------------

<div style="background: rgb(255,100,100); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 6 (Optional Bonus Question)</h3>
</div>

> **WARNING: EXTREME DIFFICULTY**<br>
> This question is optional, and frankly, not advised.<br>
> If attempting this question, be prepared to spend multiple hours or even days.<br>
> Successfully completing this question awards bonus marks which can be used to fill-in lost marks in other questions (including the short answer section). <br>
> God have mercy on any soul who ventures past this point. <br>

<br>

In Question 5 we implemented a simplified version of the bubble collapsing process. <br>
We deliberately ignored situations such as b) and c) in the figure below, where existing nodes on the consensus path need to be split. <br>
Your task is to re-implement the bubble collapsing step to handle these situations.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/collapse_hard.png" width="550">

Hint (from the Velvet publication). <br>
"""<br>
*To merge two paths, Tour Bus creates a chain of markers along both of them, node by node. The paths are merged progressively from one end to the next. At each step, the first unmapped minority node is compared to the corresponding majority consensus node, using the local sequence alignment produced previously. All the information attached to that node, including coverage, sequence identifiers, and arcs, is then mapped accordingly onto the majority node. The presence of markers allows Tour Bus to dynamically modify the marked path as it corrects the graph.*<br>
"""




<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 6</h3>

<b>Challenge:</b> Write a function which collapses the graph bubble, handling the situations above. <br>

- [ ] Input: G (networkx.DiGraph); consensus_path (list of str); minority_path (list of str).
- [ ] Ouput: Updated graph with the bubble removed (networkx.DiGraph)
    
</div>

In [ ]:
# -- OPTIONAL GRADED CELL (3 bonus marks) - complete this cell (if you dare) --

def collapse_hard(G: nx.DiGraph, consensus_path: List[str], minority_path: List[str]) -> List:
    """
    Collapses a bubble in flexible manner.
    Returns the simplified graph G with bubble removed.
    """
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - DO NOT ALTER.
k = 5
G = load_graph(NODELIST, EDGELIST)

##############################################################################
### Modifying previous graph to emulate situation b) in the figure above.  ###
##############################################################################
# new 'BC' node
l_kmer = G.edges['A', 'B']['kmer']
r_kmer = G.edges['C', 'D']['kmer']
node_bc_seq = G.nodes['B']['sequence'][:-k+2] + G.nodes['C']['sequence']
# remove nodes
for node in ['B', 'C', 'C``D`', 'E']:
    G.remove_node(node)
# add new 'BC' node and edges
G.add_node('BC', sequence=node_bc_seq)
G.add_edge('A', 'BC', kmer=l_kmer)
G.add_edge('BC', 'D', kmer=r_kmer)

####################
### Single test. ###
####################
consensus_path = ['A', 'BC', 'D']
minority_path = ['A', 'B`', 'C`', 'D']
G = collapse_hard(G, consensus_path, minority_path)

print("expected nodes: ['AB', 'CD', 'X']")
print(f"actual nodes:   {sorted(G.nodes())}")
edge_ok = True if G.has_edge('AB', 'CD') and G.edges['AB', 'CD']['kmer'] == 'look_' else False
print(f"found edge AB->CD with kmer='look_': {edge_ok}")


<div style="color: rgb(0,0,0); background: #0096FF; border: solid 1px rgb(0,0,0); padding: 10px;">

<h1>Section 2: Short-Answer Questions</h1>

</div>


### Introduction

Your lab group is investigating protein abundance across different tissues extracted from Barramundi fish. <br>
1000 samples were obtained and analysed, covering all major organs, albeit in an unbalanced manner (some organs have more samples than others). <br>
You plan to use dimensionality reduction and clustering to visualise and interpret the data, then report any findings to the scientific community.



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 7</h3>

(Short Answer Question)<br>
(2 marks, max 300 words)

<div class="alert alert-info">

**Visualising the data**

Approach [1 mark]
- Outline an approach, incorporating a dimensionality reduction technique (eg PCA, t-SNE, UMAP), to visualize the protein abundance profile of these samples.
- Write your answer using dot-point notation (rather than pseudocode) documenting all major steps.

Description [1 mark]
- Comment on your approach, explaining how common issues associated with your strategy have been handled, and how the choice of parameters may affect the output visualisation.
- Write your answer as a single paragraph.

</div>


YOUR ANSWER HERE

Approach

*   Normalize or scale protein abundance values across samples to ensure uniformity and comparability.
*   Apply a dimensionality reduction technique such as PCA to the normalized dataset to condense the high-dimensional data into the top 2-3 components that capture most of the variance.
*   Create a 2D or 3D scatter plot using the principal components obtained from PCA, where each point represents a sample. Use different colors or markers to represent different tissue types.
*   Optionally, overlay clustering results (e.g., from K-means) on the plot to visualize groups of samples with similar protein abundance profiles.
*   Label axes with the percentage of variance explained by each principal component to help interpret the visualized data patterns.

Description

 The visualization approach employs PCA to create a reduced representation of the high-dimensional dataset, making it easier to identify patterns. By visualizing the top components, we capture the most important variance in the data, simplifying interpretation. The choice of PCA helps reduce noise and highlight key structures, while axis labels provide context for data spread. The output scatter plot offers a visual summary of the dataset, revealing clusters, outliers, and potential correlations between tissue samples based on protein abundance.

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 8</h3>

(Short Answer Question)<br>
(3 marks, max 300 words)

<div class="alert alert-info">

**Clustering**
    
Approach [2 marks]
- Outline an approach, using an unsupervised clustering algorithm, to identify tissues which have similar protein abundance profiles.
- Write your answer using dot-point notation (rather than pseudocode) documenting all major steps.

Description [1 mark]
- Indicate whether the raw data or a dimensionality-reduced representation of the data would be used as input to your approach.
- Comment on how outliers (eg caused by microbial contamination of samples) or human error (eg mislabeled samples) are handled.
- Describe the outputs of your approach, indicating how they can be interpreted to satisfy our goal.
- Write your answer as a single paragraph.

</div>


YOUR ANSWER HERE

Approach
*   Normalize or scale the protein abundance values to remove the impact of differing scales among features.
*   Apply a dimensionality reduction technique like PCA to simplify the dataset and highlight significant patterns, reducing the noise.
*   Choose an unsupervised clustering algorithm:
	•	Use K-means clustering if you have an estimated number of clusters, utilizing the elbow method to determine the optimal cluster count.
	•	Use DBSCAN to find clusters of varying densities and handle outliers effectively without requiring a preset number of clusters.
*   Apply the selected clustering algorithm to the reduced data to identify groups of tissue samples with similar protein abundance profiles.
*   Visualize the clusters using scatter plots (for K-means) or density-based plots (for DBSCAN), employing different colors to represent each cluster.
*   Analyze cluster composition and potential outliers to draw insights into tissue similarities.

Description

This clustering approach uses a dimensionality-reduced representation of the data, such as PCA, to streamline clustering and focus on the most informative features. Outliers, caused by contamination or mislabeling, are addressed naturally by DBSCAN, which marks sparse regions as noise, or minimized in K-means by using centroid-based grouping. The clustering output reveals groups of tissue samples with similar protein abundance, which can be interpreted to explore tissue-specific protein patterns, helping to achieve the study’s goals of understanding tissue similarities and variations.

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>End of Assignment.</h1>
    
</div>